## Config

In [1]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

# Filepaths
artistsKnownPath = os.path.join('data', 'artistsKnown.csv')

# Set max age for new artist songs
maxAge = 5

## Log In

In [2]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Collect new artists

In [4]:
# This block can be commented out once it has been run once. It does not update each year.

# # Collect names of new artists from Music Festival Wizard
# artistsNew = fetchMusicFestivalWizard()
# # Filter out known artists who we have already listened to
# artistsNew = artistsNew.loc[~artistsNew['artistName'].isin(artistsKnown['artistName'])]
# artistsNew = artistsNew.head()
# # Search spotify for artistIDs
# artistsNew['artistID'] = artistsNew.apply(findArtistIDs, credentials=credentials, axis=1)
# # Filter out if artistID was known
# artistsNew = artistsNew.loc[~artistsNew['artistID'].isin(artistsKnown['artistID'])]
# artistsNew

In [ ]:
# Website with a list of festivals
# https://www.thrillist.com/news/nation/music-festival-lineups-tickets-us-guide

In [3]:
# Print the playlist IDs so user can select an id to add to "New Artists to Explore"
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from collections import Counter
import pandas as pd
import numpy as np
from requests_html import HTMLSession
import tekore as tk

spotify, userID, user_token = credentials
with spotify.token_as(user_token):
    # Check if playlist already exists; if not create a new one
    userPlaylists = pd.DataFrame([{'id':x.id, 'name':x.name} for x in spotify.playlists(userID, limit=40).items])

userPlaylists

,id,name
0,0Ryc3JAo4ZLiJMtq1o2v6B,Hinterland 2024
1,4gYPuLx0d87tkSGacyBWJP,Lollapalooza Chicago 2024
2,4BIYTsJ4nDunOKVCkNEUmR,Pitchfork Music Festival 2024
3,5jPqT5VZ707FchYgLyBJjx,2024 | Under The Big Sky Festival
4,4LojrrODypV6Vmh1dXi6Vx,Summerfest 2024
5,3hucYsCUS3jPd5V2bIusA8,Bonnaroo 2024 Official Playlist
6,1rrgRdG05sm2yLpXKu6qI2,Official 2024 Playlist
7,2m52nz6kWjzJg4XrtAJx0N,Boston Calling 2024
8,4m1Y98FdlAEkpsflEvjMHp,BottleRock 2024
9,56vmmZHQo2ynL8P0QepLBq,LIB 2024 Official Playlist


## Process new artists

In [4]:
# Read in known artists
artistsKnown = pd.read_csv(artistsKnownPath)

In [9]:
# Collect artists from an existing favorited playlist
playlistToFetch = '3hucYsCUS3jPd5V2bIusA8' #getPlaylistID(credentials, 'New Artists to Explore')
artistsNew = fetchTracksFromPlaylist(credentials, playlistToFetch)
artistsNew = artistsNew[['artistName', 'artistID']].drop_duplicates('artistID', ignore_index=True).reset_index(drop=True)
# Filter out if artistID was known
artistsNew = artistsNew.loc[~artistsNew['artistID'].isin(artistsKnown['artistID'])]
artistsNew

,artistName,artistID


## Get top songs

In [29]:
# Get their top three songs in last 5 years
songURIs = fetchTopSongs(credentials, artistsNew['artistID'], maxAge=maxAge)
# How many songs are in this round?
print(f"There are {len(songURIs)} songs about to be added")

There are 45 songs about to be added


## Put songs in playlist

In [30]:
# Warning: This will make a new playlist if it does not already exist
playlistID = getPlaylistID(credentials, 'Summer - Brand New Artists')
# Add songs to the playlist
addToPlaylist(credentials, songURIs, playlistID)

## Update the list of known artists

In [32]:
# Append artistsNew to artistsKnown.csv 
allArtists = pd.concat([artistsKnown, artistsNew], axis=0, ignore_index=True)
allArtists.to_csv(artistsKnownPath, index=False)

## Create playlist to drop good songs into

In [21]:
# Cull exploreArtists into ArtistHasPotential playlist
winnersPlaylist = getPlaylistID(credentials, 'Summer - New Artists to Explore Further')
print(f'Now drop songs you like into this playlist: {winnersPlaylist}')

Now drop songs you like into this playlist: 0Yj1Gqn15HdcDNcV5XJqVm
